In [141]:
from langchain.agents import create_openapi_agent
from langchain.agents.agent_toolkits import OpenAPIToolkit
from langchain.llms.openai import OpenAI
from langchain.requests import TextRequestsWrapper
from langchain.tools.json.tool import JsonSpec

import json, yaml

In [142]:
with open("openapi.yaml") as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
json_spec = JsonSpec(dict_=data, max_value_length=4000)

In [143]:
def get_api_operation_by_id(json_spec, operation_id):
    paths = json_spec.dict_.get("paths", {})
    for path, methods in paths.items():
        for method, operation in methods.items():
            if operation.get("operationId") == operation_id:
                return operation, method

    # If the operation ID is not found, return None
    return None


In [144]:
api_operation, method=get_api_operation_by_id(json_spec, "get-multiple-albums")

In [145]:
print(api_operation, method)

{'description': 'Get Spotify catalog information for multiple albums identified by their Spotify IDs.\n', 'operationId': 'get-multiple-albums', 'parameters': [{'$ref': '#/components/parameters/QueryAlbumIds'}, {'$ref': '#/components/parameters/QueryMarket'}], 'responses': {'200': {'$ref': '#/components/responses/ManyAlbums'}, '401': {'$ref': '#/components/responses/Unauthorized'}, '403': {'$ref': '#/components/responses/Forbidden'}, '429': {'$ref': '#/components/responses/TooManyRequests'}}, 'security': [{'oauth_2_0': []}], 'summary': 'Get Several Albums\n', 'tags': ['Albums'], 'x-spotify-docs-console-url': '/console/get-several-albums/', 'x-spotify-docs-endpoint-name': 'Get Multiple Albums', 'x-spotify-policy-list': {'$ref': '#/components/x-spotify-policy/metadataPolicyList'}} get


In [146]:
import re

def resolve_refs(input_dict, json_spec):
    # Check if the input_dict is a dictionary and contains a '$ref' key
    if isinstance(input_dict, dict) and '$ref' in input_dict:
        ref_value = input_dict['$ref']
        
        # Use regular expression to extract the schema name
        match = re.match(r'#/components/schemas/(\w+)', ref_value)
        if match:
            schema_name = match.group(1)
            
            # Try to find the corresponding schema in the json_spec dictionary
            schema = json_spec.dict_.get("components", {}).get("schemas", {}).get(schema_name)
            
            # If a matching schema is found, replace the '$ref' key with the schema
            if schema:
                return schema
        
    # Recursively process nested dictionaries and lists
    if isinstance(input_dict, dict):
        for key, value in input_dict.items():
            input_dict[key] = resolve_refs(value, json_spec)
    elif isinstance(input_dict, list):
        for i, item in enumerate(input_dict):
            input_dict[i] = resolve_refs(item, json_spec)
    
    return input_dict

In [147]:
def process_operation(method, api_operation, json_spec):
    if method == "get":
        return api_operation
    else:
        input_dict = api_operation["requestBody"]["content"]["application/json"]["schema"]
        json_spec = resolve_refs(json_spec.dict_)
        result = resolve_refs(input_dict, json_spec)
        return result

In [148]:
isolated_request = process_operation(method, api_operation, json_spec)

In [149]:
json.loads(json.dumps(result, indent=4))

{'description': 'Get Spotify catalog information for multiple albums identified by their Spotify IDs.\n',
 'operationId': 'get-multiple-albums',
 'parameters': [{'in': 'query',
   'name': 'ids',
   'required': True,
   'schema': {'description': 'A comma-separated list of the [Spotify IDs](/documentation/web-api/concepts/spotify-uris-ids) for the albums. Maximum: 20 IDs.\n',
    'example': '382ObEPsp2rxGrnsizN5TX,1A2GTWGtFfWp7KSQTwWOyo,2noRn2Aes5aoNVsU6iWThc',
    'title': 'Spotify Album IDs',
    'type': 'string'}},
  {'in': 'query',
   'name': 'market',
   'required': False,
   'schema': {'description': 'An [ISO 3166-1 alpha-2 country code](https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2).\n  If a country code is specified, only content that is available in that market will be returned.<br/>\n  If a valid user access token is specified in the request header, the country associated with\n  the user account will take priority over this parameter.<br/>\n  _**Note**: If neither market or

In [150]:
isolated_request["parameters"]

[{'$ref': '#/components/parameters/QueryAlbumIds'},
 {'$ref': '#/components/parameters/QueryMarket'}]

In [151]:
import re

# ref_list -> [{'$ref': '#/components/parameters/QueryAlbumIds'},
#  {'$ref': '#/components/parameters/QueryMarket'}]

def hydrateParams(json_spec, ref_list):
    last_portion_list = []
    
    for ref in ref_list:
        match = re.search(r'/([^/]+)$', ref['$ref'])
        if match:
            last_portion_list.append(json_spec["components"]["parameters"][match.group(1)])
    
    return last_portion_list

In [153]:
hydrateParams()

{'description': 'Get Spotify catalog information for multiple albums identified by their Spotify IDs.\n',
 'operationId': 'get-multiple-albums',
 'parameters': [{'in': 'query',
   'name': 'ids',
   'required': True,
   'schema': {'description': 'A comma-separated list of the [Spotify IDs](/documentation/web-api/concepts/spotify-uris-ids) for the albums. Maximum: 20 IDs.\n',
    'example': '382ObEPsp2rxGrnsizN5TX,1A2GTWGtFfWp7KSQTwWOyo,2noRn2Aes5aoNVsU6iWThc',
    'title': 'Spotify Album IDs',
    'type': 'string'}},
  {'in': 'query',
   'name': 'market',
   'required': False,
   'schema': {'description': 'An [ISO 3166-1 alpha-2 country code](https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2).\n  If a country code is specified, only content that is available in that market will be returned.<br/>\n  If a valid user access token is specified in the request header, the country associated with\n  the user account will take priority over this parameter.<br/>\n  _**Note**: If neither market or